# Using Token Predictor

Using the token predictor, we can predict the token usage of an operation before actually performing it.
We do this with the MockLLMPredictor class, see below.

In [ ]:
# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = "INSERT OPENAI KEY"

In [ ]:
from gpt_index import GPTTreeIndex, SimpleDirectoryReader, LLMPredictor, GPTListIndex, GPTKeywordTableIndex
from gpt_index.token_predictor.mock_chain_wrapper import MockLLMPredictor
from gpt_index.prompts.base import Prompt
from IPython.display import Markdown, display

### Predicting Usage of GPT Tree Index

Here we predict usage of GPTTreeIndex during index construction and querying, without making any LLM calls.

NOTE: Predicting query usage before tree is built is only possible with GPTTreeIndex due to the nature of tree traversal. Results will be more accurate if GPTTreeIndex is actually built beforehand.

In [2]:
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [3]:
llm_predictor = MockLLMPredictor(max_tokens=256)

In [4]:
index = GPTTreeIndex(documents, llm_predictor=llm_predictor)

start token ct: 0
> Building index from nodes: 5 chunks
0/57
10/57
20/57
30/57
40/57
50/57
end token ct: 19495
> [build_index_from_documents] Total token usage: 19495 tokens


In [5]:
# default query
response = index.query("What did the author do growing up?")
print(llm_predictor.last_token_usage)

start token ct: 19495
> Starting query: What did the author do growing up?
>[Level 0] Selected node: [1]/[1]
>[Level 1] Selected node: [1]/[1]
end token ct: 24988
> [query] Total token usage: 5493 tokens
5493


### Predicting Usage of GPT Keyword Table Index Query

Here we build a real keyword table index over the data, but then predict query usage.

In [6]:
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()
index = GPTKeywordTableIndex.load_from_disk('../paul_graham_essay/index_table.json')

In [7]:
llm_predictor = MockLLMPredictor(max_tokens=256)
response = index.query("What did the author do after his time at Y Combinator?", llm_predictor=llm_predictor)
print(llm_predictor.last_token_usage)

start token ct: 0
> Starting query: What did the author do after his time at Y Combinator?
query keywords: ['the', 'at', 'y', 'after', 'time', 'combinator', 'author', 'do', 'his', 'did', 'what']
Extracted keywords: ['combinator']
> Querying with idx: 3483810247393006047: of 2016 we moved to England. We wanted our kids...
> Querying with idx: 7597483754542696814: people edit code on our server through the brow...
> Querying with idx: 7572417251450701751: invited about 20 of the 225 groups to interview...
end token ct: 11313
> [query] Total token usage: 11313 tokens
11313


### Predicting Usage of GPT List Index Query

Here we build a real list index over the data, but then predict query usage.

In [8]:
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()
index = GPTListIndex.load_from_disk('../paul_graham_essay/index_list.json')

In [13]:
llm_predictor = MockLLMPredictor(max_tokens=256)
response = index.query("What did the author do after his time at Y Combinator?", llm_predictor=llm_predictor)

start token ct: 0
> Starting query: What did the author do after his time at Y Combinator?
end token ct: 23941
> [query] Total token usage: 23941 tokens


In [14]:
print(llm_predictor.last_token_usage)

23941
